In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Admin\\anaconda3\\envs\\SparkEnvironment\\Lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .appName("ErrorHandlingApp")
    .master("local[4]")
    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.sql.adaptive.enabled", "false")
    .getOrCreate()
)

sc = spark.sparkContext

spark

In [5]:
#ByDefault spark follow Permissive mode

rateCodesJsonDf = (
    spark
    .read
    .option("columnNameOfCorruptRecord", "corruptData")
    .json("C:\DataFiles\RateCodes.json")
)
rateCodesJsonDf.show(truncate = False)

+---------------+----------+-----------------------------------------+
|RateCode       |RateCodeID|corruptData                              |
+---------------+----------+-----------------------------------------+
|Standard rate  |1         |NULL                                     |
|JFK            |2         |NULL                                     |
|Newark         |3         |NULL                                     |
|NULL           |NULL      |{"RateCodeID":},"RateCode":"Westchester"}|
|Negotiated fare|5         |NULL                                     |
|Group ride     |6         |NULL                                     |
+---------------+----------+-----------------------------------------+



In [6]:
rateCodesJsonDf = (
    spark
    .read
    .option("mode", "DROPMALFORMED")
    .json("C:\DataFiles\RateCodes.json")
)
rateCodesJsonDf.show(truncate = False)

+---------------+----------+
|RateCode       |RateCodeID|
+---------------+----------+
|Standard rate  |1         |
|JFK            |2         |
|Newark         |3         |
|Negotiated fare|5         |
|Group ride     |6         |
+---------------+----------+



In [7]:
rateCodesJsonDf = (
    spark
    .read
    .option("mode", "FAILFAST")
    .json("C:\DataFiles\RateCodes.json")
)
rateCodesJsonDf.show(truncate = False)

Py4JJavaError: An error occurred while calling o53.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (192.168.86.219 executor driver): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInSchemaInferenceError(QueryExecutionErrors.scala:1520)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.handleJsonErrorsByParseMode(JsonInferSchema.scala:64)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema.$anonfun$infer$2(JsonInferSchema.scala:93)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.for

In [13]:
rateCodesJsonDf = (
    spark
    .read
    .option("header", "true")
    .option("inferschema", "true")
    .csv("C:\DataFiles\RateCodes.csv")
)
rateCodesJsonDf.show(truncate = False)

+-----------+---------------+----------+
|RateCodeID |RateCode       |IsApproved|
+-----------+---------------+----------+
|1          |Standard Rate  |Yes       |
|2          |JFK            |Yes       |
|3          |Newark         |Yes       |
|4          |NULL           |NULL      |
|Westchester|Yes            |NULL      |
|5          |Negotiated fare|NULL      |
|6          |GroupRide      |Yes       |
+-----------+---------------+----------+

